In [0]:
!pip install mxnet-cu100 gluoncv

In [0]:
import cv2
import pickle
from google.colab.patches import cv2_imshow
import numpy as np
import mxnet as mx
from datetime import datetime
from matplotlib import pyplot as plt
from gluoncv import model_zoo, data, utils
from gluoncv.data.transforms.pose import detector_to_simple_pose, heatmap_to_coord

### Utils

In [0]:
def resize(image, width=None, height=None, inter=cv2.INTER_AREA):
    # initialize the dimensions of the image to be resized and
    # grab the image size
    dim = None
    (h, w) = image.shape[:2]

    # if both the width and height are None, then return the
    # original image
    if width is None and height is None:
        return image

    if width and height:
        dim = (width, height)
    elif width is None:
        # calculate the ratio of the height and construct the dimensions
        r = height / float(h)
        dim = (int(w * r), height)
    elif height is None:
        # calculate the ratio of the width and construct the dimensions
        r = width / float(w)
        dim = (width, int(h * r))

    # resize the image
    resized = cv2.resize(image, dim, interpolation=inter)

    # return the resized image
    return resized

### Example

Using sample image with pre-trained Yolo3 model and pre-trained SimplePoseNet.

In [0]:
detector = model_zoo.get_model('yolo3_mobilenet1.0_coco', pretrained=True)
pose_net = model_zoo.get_model('simple_pose_resnet18_v1b', pretrained=True)

# Note that we can reset the classes of the detector to only include
# human, so that the NMS process is faster.

detector.reset_class(["person"], reuse_weights=['person'])

im_fname = utils.download('https://github.com/dmlc/web-data/blob/master/' +
                          'gluoncv/pose/soccer.png?raw=true',
                          path='soccer.png')
x, img = data.transforms.presets.ssd.load_test(im_fname, short=512)
print('Shape of pre-processed image:', x.shape)

class_IDs, scores, bounding_boxs = detector(x)

pose_input, upscale_bbox = detector_to_simple_pose(img, class_IDs, scores, bounding_boxs)

predicted_heatmap = pose_net(pose_input)
pred_coords, confidence = heatmap_to_coord(predicted_heatmap, upscale_bbox)

ax = utils.viz.plot_keypoints(img, pred_coords, confidence,
                              class_IDs, bounding_boxs, scores,
                              box_thresh=0.5, keypoint_thresh=0.2)
plt.show()

### Custom Example

Using sample image with fine-tunned Detectron2 model and pre-trained SimplePoseNet.

In [0]:
pose_net = model_zoo.get_model('simple_pose_resnet18_v1b', pretrained=True)

In [0]:
img_filename = '/content/drive/My Drive/data/detectron/tolpum_0-15_130.jpg'
img = cv2.imread(img_filename)
img_resized = resize(img, width=1024)
print(img_resized.shape)

## These are the detections at the original image size
detections = [
    [634.0, 770.0, 109.0, 196.0],
    [976.0, 462.0, 61.0, 131.0],
    [113.0, 230.0, 71.0, 103.0],
    [216.0, 488.0, 81.0, 152.0]
]

for detection in detections:
    x1, y1, width, height = detection
    x2 = x1 + width
    y2 = y1 + height

    ## Since we resized the image, we also need to adapt the detections
    x1 = (img_resized.shape[0] * x1) / img.shape[0]
    x2 = (img_resized.shape[0] * x2) / img.shape[0]
    y1 = (img_resized.shape[1] * y1) / img.shape[1]
    y2 = (img_resized.shape[1] * y2) / img.shape[1]

    print(x1, y1, x2, y2)

    cv2.rectangle(img_resized, (int(x1), int(y1)), (int(x2), int(y2)), (60, 71, 222), 5)
    break

cv2_imshow(img_resized)

In [0]:
## For some reason, it expects as input the following:
## custom_class_IDs with shape (1, 100, 1)
## custom_scores with shape (1, 100, 1)
## custom_bounding_boxs with shape (1, 100, 4)

# Represents the ClassID for each bbox (in this case just one class with an ID equal to 0), while the rest is filled up with -1
custom_class_IDs = [0]
N = 100 - len(custom_class_IDs)
custom_class_IDs = np.pad(custom_class_IDs, (0, N), 'constant', constant_values=(-1))
custom_class_IDs = np.expand_dims(custom_class_IDs, axis=1)
custom_class_IDs = np.expand_dims(custom_class_IDs, axis=0)
custom_class_IDs = mx.nd.array(custom_class_IDs)

# Represents Score for each bbox (in this case juse one score), while the rest is filled up with -1
custom_scores = [0.95827]
N = 100 - len(custom_scores)
custom_scores = np.pad(custom_scores, (0, N), 'constant', constant_values=(-1))
custom_scores = np.expand_dims(custom_scores, axis=1)
custom_scores = np.expand_dims(custom_scores, axis=0)
custom_scores = mx.nd.array(custom_scores)

# Represents the bbox for each object (in this case juse one bbox), while the rest is filled up with -1
custom_bounding_boxs = [[338.1333333333333, 410.6666666666667, 396.26666666666665, 515.2]]
N = 100 - len(custom_bounding_boxs)
custom_bounding_boxs = np.pad(custom_bounding_boxs, ((0, N), (0, 0)), 'constant', constant_values=(-1))
custom_bounding_boxs = np.expand_dims(custom_bounding_boxs, axis=0)
custom_bounding_boxs = mx.nd.array(custom_bounding_boxs)

In [0]:
## Optional but useful to compare shapes/types between first example and second example
assert class_IDs.shape == custom_class_IDs.shape and type(class_IDs) == type(custom_class_IDs)
assert scores.shape == custom_scores.shape and type(scores) == type(custom_scores)
assert bounding_boxs.shape == custom_bounding_boxs.shape and type(bounding_boxs) == type(custom_bounding_boxs)

In [0]:
img_filename = '/content/drive/My Drive/data/detectron/tolpum_0-15_130.jpg'

img = cv2.imread(img_filename)
img_resized = resize(img, width=1024)
print(img_resized.shape)

pose_input, upscale_bbox = detector_to_simple_pose(img_resized, custom_class_IDs, custom_scores, custom_bounding_boxs)
predicted_heatmap = pose_net(pose_input)
pred_coords, confidence = heatmap_to_coord(predicted_heatmap, upscale_bbox)

res = utils.viz.cv_plot_keypoints(img_resized, pred_coords, confidence,
                              custom_class_IDs, custom_bounding_boxs, custom_scores,
                              box_thresh=0.5, keypoint_thresh=0.2)

cv2_imshow(res)

### Video

In [0]:
## Draw Pose
def draw_keypoints(img, coords, confidence, class_ids, bboxes, scores, box_thresh=0.5, keypoint_thresh=0.2, scale=1.0, **kwargs):

    custom_colors = [(255, 255, 255), (235, 208, 52), (8, 0, 255)]

    if isinstance(img, mx.nd.NDArray):
        img = img.asnumpy()
    if isinstance(coords, mx.nd.NDArray):
        coords = coords.asnumpy()
    if isinstance(class_ids, mx.nd.NDArray):
        class_ids = class_ids.asnumpy()
    if isinstance(bboxes, mx.nd.NDArray):
        bboxes = bboxes.asnumpy()
    if isinstance(scores, mx.nd.NDArray):
        scores = scores.asnumpy()
    if isinstance(confidence, mx.nd.NDArray):
        confidence = confidence.asnumpy()

    custom_colors = [(255, 255, 255), (235, 208, 52), (8, 0, 255)]

    joint_visible = confidence[:, :, 0] > keypoint_thresh
    joint_pairs = [[0, 1], [1, 3], [0, 2], [2, 4],
                    [5, 6], [5, 7], [7, 9], [6, 8], [8, 10],
                    [5, 11], [6, 12], [11, 12],
                    [11, 13], [12, 14], [13, 15], [14, 16]]

    colormap_index = np.linspace(0, 1, len(joint_pairs))
    coords *= scale
    for i in range(coords.shape[0]):
        pts = coords[i]
        custom_color = custom_colors[class_ids[i]]
        for cm_ind, jp in zip(colormap_index, joint_pairs):
            if joint_visible[i, jp[0]] and joint_visible[i, jp[1]]:
                if class_ids[i] == 0:
                    cm_color = tuple([int(x * 255) for x in plt.cm.hot(cm_ind)[:3]])
                else:
                    cm_color = tuple([int(x * 255) for x in plt.cm.cool(cm_ind)[:3]])
                pt1 = (int(pts[jp, 0][0]), int(pts[jp, 1][0]))
                pt2 = (int(pts[jp, 0][1]), int(pts[jp, 1][1]))
                cv2.line(img, pt1, pt2, cm_color, 3)

Using Detections from Detectron2

In [0]:
input_video_filename = '/content/drive/My Drive/data/detectron/TolucaVSPumas_sample.mp4'
input_video = cv2.VideoCapture(input_video_filename)

with open('/content/drive/My Drive/data/detectron/team_detections.pickle', 'rb') as fp:
    frames_detections = pickle.load(fp)

num_frames = int(input_video.get(cv2.CAP_PROP_FRAME_COUNT))
width = int(input_video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(input_video.get(cv2.CAP_PROP_FRAME_HEIGHT))
frames_per_second = input_video.get(cv2.CAP_PROP_FPS)
num_frames = int(input_video.get(cv2.CAP_PROP_FRAME_COUNT))

output_video_filename = '/content/pose_estimation.mp4'
output_video = cv2.VideoWriter(filename=output_video_filename, 
                               fourcc=cv2.VideoWriter_fourcc(*'mp4v'),
                               fps=float(frames_per_second),
                               frameSize=(1024, 576), ## (width, height) -- MAKE SURE TO MODIFY DEPENDING ON RESIZE
                               isColor=True)

i = 0
while input_video.isOpened():

    if i % 60 == 0:
        prog = round(i / num_frames, 2) * 100
        print('{} | {}%'.format(datetime.now(), prog))

    ret, frame = input_video.read()
    if ret:

        frame_resized = resize(frame, width=1024)
        
        for frame_detections in frames_detections:
            if frame_detections[0] == i:

                number_detections = len(frame_detections[1])
                N = 100 - number_detections

                # Since we know beforehand they all belong to the same class, we can do the following:
                # Note that the PoseEstimation only accepts a ClassID = 0
                class_IDs = np.zeros(number_detections) # frame_detections[3]
                class_IDs = np.pad(class_IDs, (0, N), 'constant', constant_values=(-1))
                class_IDs = np.expand_dims(class_IDs, axis=1)
                class_IDs = np.expand_dims(class_IDs, axis=0)
                class_IDs = mx.nd.array(class_IDs)
                
                # Since we've already filtered the detections by score, we can do the following:
                scores = frame_detections[2]
                scores = np.pad(scores, (0, N), 'constant', constant_values=(-1))
                scores = np.expand_dims(scores, axis=1)
                scores = np.expand_dims(scores, axis=0)
                scores = mx.nd.array(scores)

                bounding_boxs = []
                for idx, detections in enumerate(frame_detections[1]):

                    x1, y1, x2, y2 = detections
                    width = x2 - x1
                    height = y2 - y1
                    x1 = (frame_resized.shape[0] * x1) / frame.shape[0]
                    x2 = (frame_resized.shape[0] * x2) / frame.shape[0]
                    y1 = (frame_resized.shape[1] * y1) / frame.shape[1]
                    y2 = (frame_resized.shape[1] * y2) / frame.shape[1]
                    bounding_boxs.append([x1, y1, x2, y2])
                    # cv2.rectangle(frame_resized, (int(x1), int(y1)), (int(x2), int(y2)), (60, 71, 222), 5)

                if bounding_boxs:
                    # Prepare bbox input size for model
                    # N = 100 - len(bounding_boxs)
                    bounding_boxs = np.pad(bounding_boxs, ((0, N), (0, 0)), 'constant', constant_values=(-1))
                    bounding_boxs = np.expand_dims(bounding_boxs, axis=0)
                    bounding_boxs = mx.nd.array(bounding_boxs)

                    # Run model
                    pose_input, upscale_bbox = detector_to_simple_pose(frame_resized, class_IDs, scores, bounding_boxs)
                    predicted_heatmap = pose_net(pose_input)
                    pred_coords, confidence = heatmap_to_coord(predicted_heatmap, upscale_bbox)

                    # Draw pose
                    # res = utils.viz.cv_plot_keypoints(frame_resized, pred_coords, confidence, class_IDs, bounding_boxs, scores)
                    draw_keypoints(frame_resized, pred_coords, confidence, frame_detections[3], bounding_boxs, scores)

                break

        output_video.write(frame_resized)
        i += 1
    else:
        break

input_video.release()
output_video.release()

Using Tracks from IOU Tracker

In [0]:
input_video_filename = '/content/drive/My Drive/data/detectron/TolucaVSPumas_sample.mp4'
input_video = cv2.VideoCapture(input_video_filename)

with open('/content/drive/My Drive/data/detectron/tracks.pickle', 'rb') as fp:
    tracks = pickle.load(fp)

num_frames = int(input_video.get(cv2.CAP_PROP_FRAME_COUNT))
width = int(input_video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(input_video.get(cv2.CAP_PROP_FRAME_HEIGHT))
frames_per_second = input_video.get(cv2.CAP_PROP_FPS)
num_frames = int(input_video.get(cv2.CAP_PROP_FRAME_COUNT))

output_video_filename = '/content/pose_estimation.mp4'
output_video = cv2.VideoWriter(filename=output_video_filename, 
                               fourcc=cv2.VideoWriter_fourcc(*'mp4v'),
                               fps=float(frames_per_second),
                               frameSize=(1024, 576), ## (width, height) -- MAKE SURE TO MODIFY DEPENDING ON RESIZE
                               isColor=True)

i = 0
while input_video.isOpened():

    if i % 60 == 0:
        prog = round(i / num_frames, 2) * 100
        print('{} | {}%'.format(datetime.now(), prog))

    ret, frame = input_video.read()
    if ret:
        
        frame_resized = resize(frame, width=1024)

        bounding_boxs = []
        classes = []
        for t in tracks:
            if t['id'] in [30, 31, 32, 33, 47, 50]:
                continue
            for bbox in t['bbox']:
                ## bbox[0] represents the frame number
                if bbox[0] == i:
                    x1, y1, x2, y2 = bbox[-1].tolist()
                    width = x2 - x1
                    height = y2 - y1
                    x1 = (frame_resized.shape[0] * x1) / frame.shape[0]
                    x2 = (frame_resized.shape[0] * x2) / frame.shape[0]
                    y1 = (frame_resized.shape[1] * y1) / frame.shape[1]
                    y2 = (frame_resized.shape[1] * y2) / frame.shape[1]
                    bounding_boxs.append([x1, y1, x2, y2])
                    classes.append(bbox[2])

        if bounding_boxs:
            number_detections = len(bounding_boxs)
            N = 100 - number_detections

            # Since we know beforehand they all belong to the same class, we can do the following:
            # Note that the PoseEstimation only accepts a ClassID = 0
            class_IDs = np.zeros(number_detections)
            class_IDs = np.pad(class_IDs, (0, N), 'constant', constant_values=(-1))
            class_IDs = np.expand_dims(class_IDs, axis=1)
            class_IDs = np.expand_dims(class_IDs, axis=0)
            class_IDs = mx.nd.array(class_IDs)
            
            # Since we've already filtered the detections by score, we can do the following:
            scores = np.ones(number_detections)
            scores = np.pad(scores, (0, N), 'constant', constant_values=(-1))
            scores = np.expand_dims(scores, axis=1)
            scores = np.expand_dims(scores, axis=0)
            scores = mx.nd.array(scores)

            # Prepare bbox input size for model
            bounding_boxs = np.pad(bounding_boxs, ((0, N), (0, 0)), 'constant', constant_values=(-1))
            bounding_boxs = np.expand_dims(bounding_boxs, axis=0)
            bounding_boxs = mx.nd.array(bounding_boxs)

            # Run model
            pose_input, upscale_bbox = detector_to_simple_pose(frame_resized, class_IDs, scores, bounding_boxs)
            predicted_heatmap = pose_net(pose_input)
            pred_coords, confidence = heatmap_to_coord(predicted_heatmap, upscale_bbox)

            # Draw pose
            # res = utils.viz.cv_plot_keypoints(frame_resized, pred_coords, confidence, class_IDs, bounding_boxs, scores)
            draw_keypoints(frame_resized, pred_coords, confidence, classes, bounding_boxs, scores)

            # cv2.imwrite('test.jpg', frame_resized)
            output_video.write(frame_resized)

        i += 1
    else:
        break

input_video.release()
output_video.release()